In [1]:
pip install "git+https://github.com/CU-ESIIL/cubedynamics.git@main"

  Cloning https://github.com/CU-ESIIL/cubedynamics.git (to revision main) to /tmp/pip-req-build-67723d23
  Running command git clone --filter=blob:none --quiet https://github.com/CU-ESIIL/cubedynamics.git /tmp/pip-req-build-67723d23
  Resolved https://github.com/CU-ESIIL/cubedynamics.git to commit b09488505648cb6be928be774f126a118b1df06c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


In [3]:
print(xr_data)


<xarray.DataArray 'ndvi' (time: 1, y: 2220, x: 2220)> Size: 20MB
dask.array<getitem, shape=(1, 2220, 2220), dtype=float32, chunksize=(1, 256, 256), chunktype=numpy.ndarray>
Coordinates: (12/14)
  * x                                        (x) float64 18kB 6.885e+05 ... 7...
  * y                                        (y) float64 18kB 4.778e+06 ... 4...
    constellation                            <U10 40B 'Sentinel 2'
    instruments                              <U3 12B 'msi'
    s2:saturated_defective_pixel_percentage  float64 8B 0.0
    sat:relative_orbit                       int64 8B 98
    ...                                       ...
    s2:product_type                          <U7 28B 'S2MSI2A'
    gsd                                      float64 8B 10.0
    proj:shape                               object 8B {10980}
    epsg                                     int64 8B 32613
    cubo:distance_from_center                (y, x) float64 39MB dask.array<chunksize=(256, 256), meta=n

In [13]:
import cubedynamics as cd
from cubedynamics import pipe, verbs as v
import rioxarray

# 1️⃣ Load NDVI cube for Pine Ridge, SD
cube = cd.ndvi(
    bbox=(-102.65, 42.95, -102.45, 43.10),
    start="2023-06-01",
    end="2025-12-31"
)

# 2️⃣ Compute mean NDVI over time
mean_ndvi = pipe(cube) | v.mean(dim="time")

# 3️⃣ Unwrap to get xarray DataArray
xr_data = mean_ndvi.unwrap()

# 4️⃣ Remove singleton time dimension
xr_2d = xr_data.squeeze("time")

# 5️⃣ Fully compute the dask array in memory
xr_2d = xr_2d.compute()

# 6️⃣ Ensure spatial dimensions and CRS are correct
xr_2d = xr_2d.rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=False)
xr_2d = xr_2d.rio.write_crs("EPSG:32613")  # UTM zone 13N

# 7️⃣ Export to GeoTIFF using windowed writing
output_path = r"C:\Users\jim\Downloads\mean_ndvi_2023_2025.tif"
xr_2d.rio.to_raster(output_path, windowed=True)

print(f"✅ Export complete! File saved to:\n{output_path}")


✅ Export complete! File saved to:
C:\Users\jim\Downloads\mean_ndvi_2023_2025.tif


In [15]:
import numpy as np
import matplotlib.pyplot as plt

# Assume xr_2d is your mean NDVI DataArray from before
data = xr_2d.values

# 1️⃣ Clip NDVI to [-1, 1] just to avoid weird colors
data_clipped = np.clip(data, -1, 1)

# 2️⃣ Apply colormap (Greens)
cmap = plt.cm.Greens

# 3️⃣ Normalize NDVI values to 0-1 for colormap
data_norm = (data_clipped + 1) / 2  # NDVI -1..1 -> 0..1

# 4️⃣ Map to RGBA colors
colored_img = cmap(data_norm)

# 5️⃣ Convert RGBA to 8-bit RGB
rgb_img = (colored_img[:, :, :3] * 255).astype(np.uint8)

# 6️⃣ Save as JPEG
from PIL import Image
img = Image.fromarray(rgb_img)
output_jpeg = r"C:\Users\jim\Downloads\mean_ndvi_2023_2025_green.jpg"
img.save(output_jpeg, quality=95)

print(f"✅ Green NDVI JPEG saved to:\n{output_jpeg}")


✅ Green NDVI JPEG saved to:
C:\Users\jim\Downloads\mean_ndvi_2023_2025_green.jpg
